In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile

In [4]:
def read_data(zip_path,file_name):
    archive = zipfile.ZipFile(zip_path, 'r')
    file = archive.open(file_name)
    df = pd.read_csv(file)
    return df

In [5]:
df = read_data("data.zip","Base.csv")
print(f"Dataset shape: {df.shape}")
df.head()

Dataset shape: (1000000, 32)


,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,AA,1059,...,0,1500.0,0,INTERNET,16.224843,linux,1,1,0,0
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,AD,1658,...,0,1500.0,0,INTERNET,3.363854,other,1,1,0,0
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,AB,1095,...,0,200.0,0,INTERNET,22.730559,windows,0,1,0,0
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,AB,3483,...,0,200.0,0,INTERNET,15.215816,linux,1,1,0,0
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,AA,2339,...,0,200.0,0,INTERNET,3.743048,other,0,1,0,0


In [6]:
TARGET = "fraud_bool"
FEATURES = df.drop(columns="fraud_bool").columns
X = df[FEATURES]
y= df[TARGET]

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train , X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,stratify=y,random_state=42)

In [9]:
cat_feat = X_train.select_dtypes("object").columns

In [10]:
X_train[cat_feat].nunique()

payment_type         5
employment_status    7
housing_status       7
source               2
device_os            5
dtype: int64

In [11]:
for col in cat_feat:
    print(X_train[col].value_counts())

payment_type
AB    333635
AA    232433
AC    226786
AD    106886
AE       260
Name: count, dtype: int64
employment_status
CA    657452
CB    124250
CF     39558
CC     34024
CD     23852
CE     20460
CG       404
Name: count, dtype: int64
housing_status
BC    334667
BB    235095
BA    152746
BE    152204
BD     23543
BF      1522
BG       223
Name: count, dtype: int64
source
INTERNET    893633
TELEAPP       6367
Name: count, dtype: int64
device_os
other        308217
linux        299721
windows      237146
macintosh     48437
x11            6479
Name: count, dtype: int64


In [12]:
fraud_data = X_train.loc[y_train[y_train == 1].index]

In [13]:
for col in cat_feat:
    print(fraud_data[col].value_counts())

payment_type
AC    3777
AB    3752
AA    1238
AD    1158
AE       1
Name: count, dtype: int64
employment_status
CA    8000
CB     856
CC     845
CD      91
CF      81
CE      46
CG       7
Name: count, dtype: int64
housing_status
BA    5740
BC    2034
BB    1407
BE     531
BD     206
BF       7
BG       1
Name: count, dtype: int64
source
INTERNET    9825
TELEAPP      101
Name: count, dtype: int64
device_os
windows      5842
other        1795
linux        1546
macintosh     669
x11            74
Name: count, dtype: int64


We can safely change some variable names to 'other'. HOCAYA SOR

In [18]:
def reduce_categories(df,col_list,cat_list):
    data = df.copy()
    for i in range(len(col_list)):
        data[col_list[i]] = data[col_list[i]].apply(lambda x:"other" if x not in cat_list[i] else x)

    return data

In [17]:
cols_to_reduce = [""]
cat_reduce_list = [[]]


device_os
other      363133
linux      299721
windows    237146
Name: count, dtype: int64